# Cover Type Prediction using ensembles

## Dataset Description
The dataset represents the data about trees which were planted in the US. The dataset consists of the information about 500000 trees. Your aim is to build  Random Forest Ensemble to predict the cover type of trees. In order to successfully complete this assignment you have to follow this algorithm:
* Load the training data
* Transform categorical features into vector representations
* Split dataset into the train and validation part
* Fit the Random Forest Ensemble into the training set
* Compare the accuracy of the fitted model with the Logistic Regression Model, which is about 0.67 for this set


If you have enough time, it will be very interesting to dig into further research through these steps:
* Determine which features are valuable for your model (calculate feature importance of your model).
* Try to reduce number of trees and see the results.
* Understand why the linear models have poor performance on this dataset.


## Loading data

Init pyspark session

In [4]:
from __future__ import division, print_function, unicode_literals # For the compatibility with Python 2

In [6]:
from pyspark.sql import SparkSession
spark_session = SparkSession.builder\
                            .enableHiveSupport()\
                            .appName("spark sql")\
                            .master("local[4]")\
                            .getOrCreate()

Load train dataset located at /data/covertype2 with at least 60 partitions (use function repartition for this case). Use option `inferSchema` to save numerical features.

In [7]:
sc = spark_session.sparkContext
data = spark_session.read.csv("/data/covertype2/train.csv", header = True, inferSchema=True)

In [13]:
# data.collect()
# http://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html
# data.head(10)
# data.select("Elevation", "Aspect").head(5)
# data.describe()
# print(data.dtypes)
# data.schema
# print(data.count())
# data.explain()
# type(data)
# data.head()

# xx = SoilIndex_data_onehot.head()
# xx.SoilVec
# xx.SoilVec.toArray()

Row(Elevation=3122, Aspect=266, Slope=10, Horizontal_Distance_To_Hydrology=433, Vertical_Distance_To_Hydrology=75, Horizontal_Distance_To_Roadways=3069, Hillshade_9am=195, Hillshade_Noon=245, Hillshade_3pm=188, Horizontal_Distance_To_Fire_Points=451, Wild_Type='Comanche', Soil_Type='Catamount family - Rock outcrop - Leighcan family complex, extremely stony.', Target=1)

## Transforming data

As you can see, there are two categorical features in dataset: 'Soil_Type' and 'Wild_Type'. You have to transform them into the vector embeddings.

First of all, you have to use StringIndexer to transform feature types to indexes

In [16]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

In [35]:
Indexer_Soil = StringIndexer(inputCol = 'Soil_Type', outputCol = "SoilIndex")
Indexer_Soil_model = Indexer_Soil.fit(data)
SoilIndex_data = Indexer_Soil_model.transform(data)

encoder_Soil = OneHotEncoder(inputCol = "SoilIndex", outputCol = "SoilVec")
encoder_Soil.setDropLast(True)
SoilIndex_data_onehot = encoder_Soil.transform(SoilIndex_data)

Index_Wild = StringIndexer(inputCol = "Wild_Type", outputCol = "WildIndex")
Index_Wild_model = Index_Wild.fit(SoilIndex_data_onehot)
SoilIndex_data_onehot_wild = Index_Wild_model.transform(SoilIndex_data_onehot)

encoder_Wild = OneHotEncoder(inputCol = "WildIndex", outputCol = "WildVec")
encoder_Wild.setDropLast(True)
data_onehot = encoder_Wild.transform(SoilIndex_data_onehot_wild)

data_onehot.head()


Row(Elevation=3122, Aspect=266, Slope=10, Horizontal_Distance_To_Hydrology=433, Vertical_Distance_To_Hydrology=75, Horizontal_Distance_To_Roadways=3069, Hillshade_9am=195, Hillshade_Noon=245, Hillshade_3pm=188, Horizontal_Distance_To_Fire_Points=451, Wild_Type='Comanche', Soil_Type='Catamount family - Rock outcrop - Leighcan family complex, extremely stony.', Target=1, SoilIndex=2.0, SoilVec=SparseVector(39, {2: 1.0}), WildIndex=1.0, WildVec=SparseVector(3, {1: 1.0}))

Apply OneHotEncoder technique to the dataset in order to get vectors for the Random Forest classification

Use the VectorAssembler technique to accumulate all features into one vector. Don't forget to use features that you have generated

In [37]:
from pyspark.ml.feature import VectorAssembler

In [38]:
vector_assembler = VectorAssembler(inputCols=['SoilVec', # feature name of Soil type encoded
                                              'WildVec', # feature name of Wild type encoded
                                              'Elevation',
                                              'Aspect',
                                              'Slope',
                                              'Horizontal_Distance_To_Hydrology',
                                              'Vertical_Distance_To_Hydrology',
                                              'Horizontal_Distance_To_Roadways',
                                              'Hillshade_9am',
                                              'Hillshade_Noon',
                                              'Hillshade_3pm',
                                              'Horizontal_Distance_To_Fire_Points'
                                              ], outputCol='features')

In [40]:
data_features = vector_assembler.transform(data_onehot)
print(data_features.head())

Row(Elevation=3122, Aspect=266, Slope=10, Horizontal_Distance_To_Hydrology=433, Vertical_Distance_To_Hydrology=75, Horizontal_Distance_To_Roadways=3069, Hillshade_9am=195, Hillshade_Noon=245, Hillshade_3pm=188, Horizontal_Distance_To_Fire_Points=451, Wild_Type='Comanche', Soil_Type='Catamount family - Rock outcrop - Leighcan family complex, extremely stony.', Target=1, SoilIndex=2.0, SoilVec=SparseVector(39, {2: 1.0}), WildIndex=1.0, WildVec=SparseVector(3, {1: 1.0}), features=SparseVector(52, {2: 1.0, 40: 1.0, 42: 3122.0, 43: 266.0, 44: 10.0, 45: 433.0, 46: 75.0, 47: 3069.0, 48: 195.0, 49: 245.0, 50: 188.0, 51: 451.0}))


## Training

Fit the Random Forest model to the train dataset. Don't forget to split dataset into two parts to check your trained models. It is desirable to use about 100 trees with depth about 7 in order to avoid wasting too much time waiting while your model will be fit to the data. Try to adjust the options 'subsamplingRate' and 'featureSubsetStrategy' to get better results

<b> Extra task.</b> Use the Cross-Validation to check your model.

In [41]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

rf = RandomForestClassifier(labelCol = "Target", featuresCol = "features", numTrees = 100, maxDepth = 7)

In [55]:
rf_model = rf.fit(data_features)

Get the feature importances of the trained model. What 5 features are the most important in the dataset?

In [58]:
xx = rf_model.featureImportances.toArray()
len(xx)

52

Apply model to the validation part of your set and get the accuracy score for the data. Use the MulticlassClassificationEvaluator class from the ml.evaluation module. 

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [66]:
predictionsandLabel = rf_model.transform(data_features)

# RandomForestClassificationModel has no attribute Predict

In [67]:
predictionsandLabel.head()

Row(Elevation=3122, Aspect=266, Slope=10, Horizontal_Distance_To_Hydrology=433, Vertical_Distance_To_Hydrology=75, Horizontal_Distance_To_Roadways=3069, Hillshade_9am=195, Hillshade_Noon=245, Hillshade_3pm=188, Horizontal_Distance_To_Fire_Points=451, Wild_Type='Comanche', Soil_Type='Catamount family - Rock outcrop - Leighcan family complex, extremely stony.', Target=1, SoilIndex=2.0, SoilVec=SparseVector(39, {2: 1.0}), WildIndex=1.0, WildVec=SparseVector(3, {1: 1.0}), features=SparseVector(52, {2: 1.0, 40: 1.0, 42: 3122.0, 43: 266.0, 44: 10.0, 45: 433.0, 46: 75.0, 47: 3069.0, 48: 195.0, 49: 245.0, 50: 188.0, 51: 451.0}), rawPrediction=DenseVector([0.0, 50.5481, 44.2126, 0.6682, 0.0046, 0.5145, 0.4268, 3.6252]), probability=DenseVector([0.0, 0.5055, 0.4421, 0.0067, 0.0, 0.0051, 0.0043, 0.0363]), prediction=1.0)

In [68]:
evaluator = MulticlassClassificationEvaluator(labelCol="Target", predictionCol="prediction", metricName="f1")

In [69]:
f1 = evaluator.evaluate(predictionsandLabel)

In [72]:
f1

0.6728425670212065

Are your results better than the results from the Logistic Regression model?

# Performing test submission

Apply the models to the test dataset.

<b>Note!</b> Dataset will be changed during the test phase. Your last cell output must be the accuracy score.

In [ ]:
# Load dataset, transform dataset

In [ ]:
# Calculate accuracy

In [ ]:
# This is last cell. The results of the evaluator must be here